In [9]:
%reload_ext autoreload
%autoreload 2
import os
import yaml
if "NeuroVoice" != os.path.basename(os.getcwd()):
    os.chdir("../")
from pipelines.train import TrainPipeline
from IPython.display import Audio, display
from models.tts.xtts_finetuner import XTTSFinetuner, OUTPUT_SAMPLE_RATE

config = yaml.safe_load(open("configs/train/divertito.yaml"))
trainpipe = TrainPipeline(config)
# trainpipe.train()

# 3. Проверяем обученную модель

In [5]:
checkpoint_path="out/divertito/xtts/best_model_v2.pth"
base_model_path = trainpipe.finetuner.load_base_model()
model = trainpipe.finetuner.load_model(
    trainpipe.device,
    os.path.join(base_model_path,"config.json"),
    checkpoint_path,
    os.path.join(base_model_path, "vocab.json"),
   os.path.join(base_model_path, "speakers_xtts.pth")
)

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]/home/egatiyatullin/repos/NeuroVoice/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 18 files: 100%|██████████| 18/18 [00:00<00:00, 402.95it/s]


In [10]:
# Generate audio
audio = trainpipe.finetuner.generate_audio(
    model,
    "Это, конечно, потешно блядь. Ты сладкий бубочка, а не уважающий себя мужчина, хахах",
    config["data"]["reference_audio"]
)
display(Audio(audio,  rate = OUTPUT_SAMPLE_RATE))

NameError: name 'OUTPUT_SAMPLE_RATE' is not defined